## Perform an EDA on the training data

- [FAIR's 2017 ACL paper](https://research.fb.com/publications/automatically-generating-rhythmic-verse-with-neural-networks/) achieved pretty good result using 34 million character set, with a 3-layer LSTM (2048-dimension hidden layer, and 128 dimension encoding). The model converged in 20 epochs.
- I think there might be something wrong with my training data.

In [6]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
from bs4 import BeautifulSoup
import os
import re
import pandas as pd
%matplotlib inline
import json
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.decomposition import NMF, LatentDirichletAllocation

In [3]:
# Read in the training data
def vocab2vec(vocab_size, vocab_length=10**7):
    f = open("Combined_String.txt", "r")
    s = f.read()
    f.close()
    D = 'abcdefghijklmnopqrstuvwxyz .,\'()1234567890";'
    res = []
    for i in range(vocab_length):
        c = s[i].lower()
        v = np.zeros((vocab_size))
        try:
            idx = D.index(c)
            v[idx] = 1
            res.append(v)
        except (ValueError, IndexError) as e:
            pass
        
        
    ret = np.array(res) # A list of shape (vocab_length,) one-hot encoded characters
    print ("shape is: {}".format(ret.shape))
    return ret

vocab2vec(40)
# Around 10 million characters.

shape is: (9919422, 40)


array([[ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       ..., 
       [ 0.,  0.,  1., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.],
       [ 0.,  0.,  0., ...,  0.,  0.,  0.]])

In [5]:
# Count the most frequent 100 words
f = open("Combined_String.txt", "r")
s = f.read().lower()
f.close()
s.replace("\t", " ")
s.replace("\n", " ")
words = s.split(" ")
print ("len(words)={}".format(len(words)))
f = open("combined_word_lists.txt", "w+")
f.write(json.dumps(words))
f.close()

len(words)=42632595


In [7]:
# What's wrong... after spliting, the length is longer than 


In [ ]:
# Perform an LDA on the data